In [ ]:
# Librerias necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# warning
import  warnings
warnings.filterwarnings("ignore")

In [ ]:
dataset=[
      ["Leche", "Huevos"],
      ["Pan", "Queso"],
      ["Leche", "Pan", "Huevos"],
      ["Pan", "Queso"],
      ["Leche","Queso","Huevos"],
      ["Pan","Queso","Huevos"],
      ["Leche","Pan","Queso","Huevos"]
]

In [ ]:
# algoritmo apriori
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

###  Soporte (Support)
$$
\text{Soporte}(A \rightarrow B) = P(A \cap B)
$$

**Interpretación:**  
Es la probabilidad de que **A y B ocurran simultáneamente** en una transacción o evento.  
Representa la frecuencia conjunta.

---

###  Confianza (Confidence)
$$
\text{Confianza}(A \rightarrow B) = P(B \mid A) = \frac{P(A \cap B)}{P(A)}
$$

**Interpretación:**  
Es la probabilidad de que **B ocurra dado que A ya ocurrió**.  
Mide la *fuerza direccional* de la asociación desde A hacia B.

---

###  Elevación (Lift)
$$
\text{Lift}(A \rightarrow B) = \frac{P(B \mid A)}{P(B)} = \frac{P(A \cap B)}{P(A) \, P(B)}
$$

**Interpretación:**  
Compara la probabilidad conjunta observada con la probabilidad esperada si A y B fueran **independientes**:
- Si \( $\text{Lift} > 1$ \): $A$ y $B$ tienen una **asociación positiva**.  
- Si \( $\text{Lift} = 1$ \): $A$ y $B$ son **independientes**.  
- Si \( $\text{Lift} < 1$ \): $A$ y $B$ tienen una **asociación negativa**.


In [ ]:
# Convertir los datos a formato canasta
te = TransactionEncoder()
# ajustar
te_ary = te.fit(dataset).transform(dataset)
# convertir a dataframe
df = pd.DataFrame(te_ary,columns=te.columns_)
df

,Huevos,Leche,Pan,Queso
0,True,True,False,False
1,False,False,True,True
2,True,True,True,False
3,False,False,True,True
4,True,True,False,True
5,True,False,True,True
6,True,True,True,True


In [ ]:
# Obtener los conjuntos frecuentes(apriori)
np.random.seed(2025)# Fijar semilla
frequent_itemsets = apriori(df, min_support=0.2, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.714286,(Huevos)
1,0.571429,(Leche)
2,0.714286,(Pan)
3,0.714286,(Queso)
4,0.571429,"(Leche, Huevos)"
5,0.428571,"(Huevos, Pan)"
6,0.428571,"(Queso, Huevos)"
7,0.285714,"(Leche, Pan)"
8,0.285714,"(Queso, Leche)"
9,0.571429,"(Queso, Pan)"


In [ ]:
# Generar reglas de asociacion
reglas=association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)
print(reglas[["antecedents","consequents","support","confidence","lift"]])

        antecedents      consequents   support  confidence      lift
0          (Huevos)          (Leche)  0.571429    0.800000  1.400000
1           (Leche)         (Huevos)  0.571429    1.000000  1.400000
2             (Pan)         (Huevos)  0.428571    0.600000  0.840000
3          (Huevos)            (Pan)  0.428571    0.600000  0.840000
4          (Huevos)          (Queso)  0.428571    0.600000  0.840000
5           (Queso)         (Huevos)  0.428571    0.600000  0.840000
6           (Leche)            (Pan)  0.285714    0.500000  0.700000
7           (Leche)          (Queso)  0.285714    0.500000  0.700000
8             (Pan)          (Queso)  0.571429    0.800000  1.120000
9           (Queso)            (Pan)  0.571429    0.800000  1.120000
10    (Pan, Huevos)          (Leche)  0.285714    0.666667  1.166667
11     (Pan, Leche)         (Huevos)  0.285714    1.000000  1.400000
12  (Huevos, Leche)            (Pan)  0.285714    0.500000  0.700000
13          (Leche)    (Pan, Huevo

| Método   | Nombre                               | Basado en                | Qué mide                                                              |
| :------- | :----------------------------------- | :----------------------- | :-------------------------------------------------------------------- |
| **UBCF** | *User-Based Collaborative Filtering* | Similitud entre usuarios | Busca usuarios con gustos similares para recomendarles ítems comunes. |
| **IBCF** | *Item-Based Collaborative Filtering* | Similitud entre ítems    | Recomienda ítems parecidos a los que el usuario ya calificó alto.     |


In [ ]:
#! pip install scikit-surprise
#! pip install "numpy<2"

In [ ]:
from surprise import Dataset, Reader, SVD, accuracy

In [ ]:
# Crear valoraciones de peliculas
ranking={
    "usuario":["A","A","A","B","B","B","C","C","C"],
    "pelicula":["pelicula1","pelicula2","pelicula3","pelicula1","pelicula2","pelicula,3","pelicula1","pelicula2","pelicula3"],
    "Valoracion":[5,4,5,1,4,5,4,3,3]}
df=pd.DataFrame(ranking)
df


In [ ]:
# convertir en formato surprise
reader=Reader(rating_scale=(1,5))
data=Dataset.load_from_df(df[["usuario","pelicula","Valoracion"]],reader=reader)
data

In [ ]:
# Data de entrenamiento
from surprise.model_selection import train_test_split
trainset,testset=train_test_split(data,test_size=0.25, random_state=2025)

In [ ]:
# modelo UBCF(Filtrado colaborativo)
from surprise import KNNBasic, accuracy
opciones={
    "name":"cosine",
    "user_based":True
}
# crear modelo
model_UBCF=KNNBasic(sim_options=opciones)

In [ ]:
# ajustemos
prediciones=model_UBCF.fit(trainset).test(testset)
accuracy.rmse(prediciones)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.2910


1.2909944487358056

In [ ]:
# Predccion de calificar un usuario
pred=model_UBCF.predict("A","pelicula3")
print(pred)

user: A          item: pelicula3  r_ui = None   est = 3.00   {'actual_k': 1, 'was_impossible': False}


In [ ]:
# Modelo IBCF(Similitud entre items)
opciones={
    "name":"cosine",
    "user_based":False
}
# crear modelo
model_IBCF=KNNBasic(sim_options=opciones)
# ajustemos
prediciones=model_IBCF.fit(trainset).test(testset)
accuracy.rmse(prediciones)


Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.9149


1.9148542155126762

In [ ]:
# Predccion de calificar un usuario
pred=model_IBCF.predict("A","pelicula3")
print(pred)

user: A          item: pelicula3  r_ui = None   est = 4.00   {'actual_k': 1, 'was_impossible': False}
